In [5]:
!pip install numpy
!pip install pillow
!pip install spicy

import tkinter as tk
from tkinter import filedialog, Menu
from PIL import Image, ImageTk
import numpy as np
from scipy.signal import convolve2d

# Variables globales
canvas = None
photo = None
img = None                  # Image PIL d'origine
nom_de_fichier = None
mat_original = None         # Image d'origine (tableau numpy)
base_mat = None             # Base utilisée pour recalculer l'image
applied_filters = []        # Liste des filtres appliqués

# Définition des groupes de filtres (pour éviter de cumuler des filtres incompatibles)
filter_groups = {
    "gris": "color",
    "vert": "color",
    "blur": "blur"
}

fenetre = tk.Tk()
fenetre.title("Modifieur d'image")

def charger():
    global photo, img, canvas, mat_original, base_mat, applied_filters, nom_de_fichier
    nom_de_fichier = filedialog.askopenfilename(
        title="Ouvrir une image",
        filetypes=[("Images", "*.jpg;*.jpeg;*.png;*.bmp;*.gif"), ("Tous", "*.*")]
    )
    if nom_de_fichier != "":
        img = Image.open(nom_de_fichier)
        mat_original = np.array(img)
        base_mat = mat_original.copy()
        applied_filters = []  # Réinitialiser la liste des filtres
        if canvas is None:
            creer_canvas(img.size[0], img.size[1])
        update_display()

def creer_canvas(largeur, hauteur):
    global canvas
    canvas = tk.Canvas(fenetre, width=largeur, height=hauteur)
    canvas.pack()

def rafraichir(image):
    """
    Met à jour l'affichage de l'image sur le canevas.
    """
    global photo, canvas
    photo = ImageTk.PhotoImage(image)
    canvas.delete("all")
    canvas.create_image(0, 0, anchor=tk.NW, image=photo)
    canvas.config(width=image.size[0], height=image.size[1])
    fenetre.pack_propagate(True)

def apply_filter_gris(image):
    """
    Applique le filtre gris sur une image PIL.
    """
    arr = np.array(image)
    coefficients = np.array([0.2125, 0.7154, 0.0721])
    gris = np.dot(arr[..., :3], coefficients)
    arr[..., :3] = gris[..., np.newaxis]
    return Image.fromarray(arr.astype(np.uint8))

def apply_filter_vert(image):
    """
    Applique le filtre vert sur une image PIL (annule les canaux rouge et bleu).
    """
    arr = np.array(image)
    arr[..., [0, 2]] = 0
    return Image.fromarray(arr.astype(np.uint8))

def box_blur(image, rayon):
    """
    Applique un flou par moyenne (box blur) sur une image PIL à l'aide de convolve2d.
    """
    if rayon == 0:
        return image
    arr = np.array(image.convert("RGB"), dtype=np.float32)
    h, w, c = arr.shape
    k = 2 * rayon + 1
    kernel = np.ones((k, k), dtype=np.float32) / (k * k)
    output = np.empty_like(arr)
    for i in range(c):
        output[:, :, i] = convolve2d(arr[:, :, i], kernel, mode="same", boundary="symm")
    output = np.clip(output, 0, 255).astype(np.uint8)
    return Image.fromarray(output)

def apply_filter_blur(image, radius):
    """
    Applique le flou sur une image PIL avec un rayon donné.
    """
    return box_blur(image, radius)

def update_display():
    """
    Repart de l'image d'origine et applique, dans l'ordre, tous les filtres enregistrés.
    """
    global base_mat, applied_filters
    result = Image.fromarray(base_mat.astype(np.uint8))
    for filt in applied_filters:
        if filt["type"] == "gris":
            result = apply_filter_gris(result)
        elif filt["type"] == "vert":
            result = apply_filter_vert(result)
        elif filt["type"] == "blur":
            radius = filt["params"].get("radius", 0)
            result = apply_filter_blur(result, radius)
    rafraichir(result)

def ajouter_filtre(filtre_type, params):
    """
    Ajoute un filtre à la liste en retirant les filtres incompatibles du même groupe.
    Pour le flou, on n'ajoute que si le rayon est > 0.
    """
    global applied_filters
    group = filter_groups.get(filtre_type)
    # Retirer les filtres déjà appliqués appartenant au même groupe
    applied_filters = [f for f in applied_filters if f["group"] != group]
    if filtre_type == "blur" and params.get("radius", 0) <= 0:
        update_display()
        return
    applied_filters.append({"type": filtre_type, "group": group, "params": params})
    update_display()

def filtre_gris():
    """
    Commande pour appliquer le filtre gris.
    """
    ajouter_filtre("gris", {})

def filtre_vert():
    """
    Commande pour appliquer le filtre vert.
    """
    ajouter_filtre("vert", {})

def ouvrir_filtre_flou():
    """
    Ouvre une fenêtre de paramètres pour définir l'intensité du flou.
    Le curseur va de 0 à 10 et se positionne sur la valeur du flou déjà appliqué (si présent).
    """
    top = tk.Toplevel(fenetre)
    top.title("Paramètres du flou (scipy)")
    
    label = tk.Label(top, text="Rayon du flou :")
    label.pack(pady=5)
    
    # Récupérer la valeur actuelle du flou s'il existe déjà
    init_val = 0
    for f in applied_filters:
        if f["type"] == "blur":
            init_val = f["params"].get("radius", 0)
            break

    scale = tk.Scale(top, from_=0, to=10, orient=tk.HORIZONTAL)
    scale.set(init_val)
    scale.pack(pady=5)
    
    def valider():
        rayon = scale.get()
        ajouter_filtre("blur", {"radius": rayon})
        top.destroy()
    
    bouton_valider = tk.Button(top, text="Valider", command=valider)
    bouton_valider.pack(pady=5)

def reinitialiser():
    """
    Réinitialise l'image en retirant tous les filtres.
    """
    global applied_filters, base_mat, mat_original
    applied_filters = []
    base_mat = mat_original.copy()
    update_display()

def sauvegarder():
    """
    Sauvegarde l'image finale (après application des filtres) dans un fichier.
    """
    global base_mat, applied_filters
    result = Image.fromarray(base_mat.astype(np.uint8))
    for filt in applied_filters:
        if filt["type"] == "gris":
            result = apply_filter_gris(result)
        elif filt["type"] == "vert":
            result = apply_filter_vert(result)
        elif filt["type"] == "blur":
            radius = filt["params"].get("radius", 0)
            result = apply_filter_blur(result, radius)
    chemin = filedialog.asksaveasfilename(
        defaultextension=".png",
        filetypes=[("PNG", "*.png"), ("JPEG", "*.jpg;*.jpeg"), ("Tous", "*.*")]
    )
    if chemin:
        result.save(chemin)

# Création du menu principal
menu_bar = Menu(fenetre)
fenetre.config(menu=menu_bar)

# Menu Fichier
menu_file = Menu(menu_bar, tearoff=0)
menu_file.add_command(label="Ouvrir", command=charger)
menu_file.add_command(label="Réinitialiser", command=reinitialiser)
menu_file.add_command(label="Enregistrer", command=sauvegarder)
menu_file.add_command(label="Quitter", command=fenetre.quit)
menu_bar.add_cascade(label="Fichier", menu=menu_file)

# Menu Effets
menu_effets = Menu(menu_bar, tearoff=0)
menu_effets.add_command(label="Filtre Gris", command=filtre_gris)
menu_effets.add_command(label="Filtre Vert", command=filtre_vert)
menu_effets.add_command(label="Filtre Flou (scipy)", command=ouvrir_filtre_flou)
menu_bar.add_cascade(label="Effets", menu=menu_effets)

fenetre.mainloop()


